# GPX HeatMap 

This will read all the gpx files within a directory, read the track_points and use them to make a heatmap.

![HeatMap output](HeatMapOutput.png "HeatMap Output")

Prior to using this, download your gpx files to a local location.  If taking lots of files from GarminConnect use https://github.com/pe-st/garmin-connect-export
This notebook then reads the list of files, opens each file into a GeoPandas DataFrame and appends the contents to a GeoDataFrame to hold all the points.
We then find the centre of all the points to centralise the map
Create HeatMap data by reading in the GeoDataFrame line by line and process the map

Set up some libraries

In [148]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point
from folium import Choropleth, Marker

Set up a function to show the map and save the html file

In [25]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

read in track_points from the gpx file into a GeoPandas dataframe

In [129]:
mydir = "/media/psf/Home/Documents/GitHub/GPX/MyActivities"

# initialise an empty dataframe called gdf_points
gdf_all = gpd.GeoDataFrame()

# find the *gpx files in mydir
import os
for file in os.listdir(mydir):
    if file.endswith(".gpx"):
        
#         Doing this in 2 steps as reading in and appending in one go didn't seem to work
        gdf = gpd.read_file(mydir + "/" + file, layer='track_points')
        gdf_all = gdf_all.append(gdf)
        
# This finds the centre lat and lon of the whole geodataframe, which I use to centre the map on opening        
start_lon = gdf_all.unary_union.centroid.x
start_lat = gdf_all.unary_union.centroid.y

gdf_all.head()

,track_fid,track_seg_id,track_seg_point_id,ele,time,magvar,geoidheight,name,cmt,desc,src,link1_href,link1_text,link1_type,link2_href,link2_text,link2_type,sym,type,fix,sat,hdop,vdop,pdop,ageofdgpsdata,dgpsid,ns3_TrackPointExtension,geometry
0,0,0,0,77.000000,2020-03-13T11:37:51,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<ns3:atemp>15.0</ns3:atemp> <ns3:hr>80</ns3:hr>,POINT (-0.93341 51.01340)
1,0,0,1,77.000000,2020-03-13T11:37:53,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<ns3:atemp>15.0</ns3:atemp> <ns3:hr>81</ns3:hr>,POINT (-0.93328 51.01341)
2,0,0,2,77.199997,2020-03-13T11:37:56,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<ns3:atemp>15.0</ns3:atemp> <ns3:hr>84</ns3:hr>,POINT (-0.93307 51.01346)
3,0,0,3,77.199997,2020-03-13T11:37:57,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<ns3:atemp>15.0</ns3:atemp> <ns3:hr>84</ns3:hr>,POINT (-0.93300 51.01347)
4,0,0,4,76.800003,2020-03-13T11:38:05,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<ns3:atemp>15.0</ns3:atemp> <ns3:hr>86</ns3:hr>,POINT (-0.93236 51.01357)


In [147]:
def add_markers(mapobj, gdf):
    heat_data = [[row.geometry.y,row.geometry.x] for index, row in gdf.iterrows()]
    HeatMap(data=heat_data, radius=12).add_to(mapobj)
    
# f = folium.Figure(height = 400)
m = folium.Map([start_lat, start_lon], zoom_start = 9)
# m.add_to(f)

# this calls the add_markers function
add_markers(m, gdf_all)

embed_map(m, "HeatMap.html")